Suppose you have a polygon class that has a verices property that needs to be defined as a sequence of Point2D intstances. So here, not only do we want the vertices attribute of our polygon to be iterable of some kind, we also want the elements to all be instances of Point2D class.
In turn we'll also want to make sure that coordinates for Point2D are non-negative integer values (as might be expected in computer screen coordinates).

In [24]:
class Int:
    def __init__(self, min_value = None, max_value = None):
        self.min_value = min_value
        self.max_value = max_value

    def __set_name__(self, owner_class, name):
        self.name = name
    
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError(f'{self.name} must be an integer')
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'{self.name} must be greater than or equal to {self.min_value}')
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f'{self.name} must be less than or equal to {self.max_value}')
        instance.__dict__[self.name] = value
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)


In [59]:
class Point2D:
    x = Int(min_value=0, max_value=100)
    y = Int(min_value=0, max_value=100)

    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __str__(self):
        return f'Point2D: ({self.x}, {self.y})'

    def __eq__(self, other):
        if not isinstance(other, Point2D):
            return NotImplemented
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))


In [26]:
import collections

class Point2DSequence:

    def __init__(self, min_length = None, max_length = None):
        self.min_length = min_length
        self.max_length = max_length

    def __set_name__(self, cls, name):
        self.name = name

    def __set__(self, instance, value):
        if not isinstance(value, collections.abc.Sequence):
            raise TypeError(f'{self.name} must be a sequence')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.name} must have at least {self.min_length} elements')
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f'{self.name} must have at most {self.max_length} elements')
        
        for index, item in enumerate(value):
            if not isinstance(item, Point2D):
                raise TypeError(f'Item at index {index} of {self.name} must be a Point2D instance')
        
        instance.__dict__[self.name] = list(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = []
        return instance.__dict__.get(self.name)


In [50]:
class Polygon:
    vertices = Point2DSequence(min_length=3)

    def __init__(self, *vertices):
        self.vertices = vertices
    
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise TypeError('Only Point2D instances can be appended')
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            raise ValueError(f'Cannot add more than {max_length} vertices')
        self.vertices.append(pt)
    
    def __len__(self):
        return len(self.vertices)
    
    def __getitem__(self, index):
        return self.vertices[index]
    
    def __iadd__(self, pt):
        self.append(pt)
        return self
    
    def __contains__(self, pt):
        return pt in self.vertices


In [51]:
class Triangle(Polygon):
    vertices = Point2DSequence(min_length=3, max_length=3)

class Rectangle(Polygon):
    vertices = Point2DSequence(min_length= 4, max_length=4)
    

In [49]:
try:
    p = Triangle(Point2D(0, 0), Point2D(50, 50), Point2D(50, 50), Point2D(100, 100))
except Exception as e:
    print(f"Error: {e}")

Error: vertices must have at most 3 elements


In [55]:
p = Polygon(Point2D(0, 0), Point2D(50, 50), Point2D(100, 100))
list(p)



[Point2D(x=0, y=0), Point2D(x=50, y=50), Point2D(x=100, y=100)]

In [ ]:
p+= Point2D(100, 100) #iadd
list(p)

[Point2D(x=0, y=0),
 Point2D(x=50, y=50),
 Point2D(x=100, y=100),
 Point2D(x=100, y=100)]

In [ ]:
Point2D(x=0, y=0) in p #contains


True